Thymio mini project

In [1]:
!pip install --upgrade tdmclient

In [1]:
# import libraries
import time
from tdmclient import ClientAsync
import numpy as np
import math

# import project files
import constants as cst
import extended_Kalman_filter as eKf
import vision
import geometry as geo


-1.5707963267948966


In [2]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node df054775-a7a5-454e-9ad3-7896a54bbcc8

In [3]:
# Set the motor speeds
async def stop_robot():
    """
    Stop the robot. Set the motor speed of each wheel to zero.
    """
    await node.set_variables({
            "motor.left.target": [0],
            "motor.right.target": [0],
        })

async def move_robot(left_speed, right_speed):
    """
    Turn the robot wheels to the given speeds.
    Parameter: - left_speed    : left wheel speed in [thymio speed]
               - right_speed   : right wheel speed in [thymio speed]
    """
    await node.set_variables({
            "motor.left.target": [left_speed],
            "motor.right.target": [right_speed],
        })

In [8]:
# Get the proximity sensor values
async def get_prox_sensors():
    """
    Get the sensor measurements of a the proximity sensors.
    Returns: -     : 
    """
    
    await node.wait_for_variables({str('prox.horizontal')})
    prox_sensors = np.asarray(list(node['prox.horizontal']))
    prox_sensors = prox_sensors[0:5]

    prox_sensors_bool = (prox_sensors >= np.ones(5) * 50) #cst.THRESHOLD_PROX_SENSOR
    detected = False
    if(any(prox_sensors_bool)):
        detected = True

    idx = np.argmax(prox_sensors)

    return prox_sensors, detected, idx

# Get the motor speed measurement values
async def get_motor_speed_meas():
    """
    Get the motor speed measurements
    Returns: -     : 
    """
    
    await node.wait_for_variables({str('motor.left.speed')})
    speed_left_meas = node['motor.left.speed']
    await node.wait_for_variables({str('motor.right.speed')})
    speed_right_meas = node['motor.right.speed']

    return speed_left_meas, speed_right_meas

In [5]:
# test thymio
# move the robot forward and get prox values.
await move_robot(50, 50)
dt = 0
old_time = time.time()
while(dt < 2):
    y,d,i = await get_prox_sensors()
    print(y)
    print(d)
    print(i)
    await client.sleep(0.1)
    dt = time.time() - old_time
await stop_robot()

[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0


In [9]:

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

intermediate_goals = [(0, 10), (10, 10)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]

Kfilter = eKf.Kalman(robot[0], robot[0], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    #prox_sensors, detected, max_index = await get_prox_sensors()
    # test if local (with proximity)
    #if(detected):
    #    global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    # print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    #robot[0] = robot[0] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.cos(robot_orientation)
    #robot[1] = robot[1] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.sin(robot_orientation)
    #robot_orientation = robot_orientation + dt * (robot_speed[0] - robot_speed[1])/(4.0*cst.WHEELS_DIST)
    #print(robot[0]," ", robot[1]," ", robot_orientation)
    
    Kfilter.dt_update(dt)
    speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    Kfilter.measurement_wheels(speed_left_meas / 18.1818, speed_right_meas / 18.1818)
    Kfilter.Kalman_filter()
    print("Kalman: ", Kfilter.Mu)

    if global_state == 'rotation':
        angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        #angle = robot_orientation - np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0])
        print("rotation")
        #print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        if(angle > 0):
            robot_speed = [-20, 20]
        else:
            robot_speed = [20, -20]
        await move_robot(robot_speed[0], robot_speed[1])
        

    elif global_state == 'translation':
        print("translation ", intermediate_goals)
        #print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        if abs((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 1:
        #if abs((intermediate_goals[0][0] - robot[0])**2 - (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop(0)
            global_state = 'rotation'
            continue
        
        # call translation fct
        robot_speed = [50, 50]
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        
        global_state = 'rotation'
    time.sleep(0.05)

    iter = iter + 1

0
Kalman:  [[0.        ]
 [0.        ]
 [0.        ]
 [1.26500279]
 [1.23750273]]
rotation
1
Kalman:  [[ 1.0941987 ]
 [ 0.        ]
 [ 0.05418388]
 [ 1.13300164]
 [-1.12200242]]
rotation
2
Kalman:  [[ 1.10554599e+00]
 [ 1.95478488e-04]
 [ 7.21346782e-02]
 [ 1.14653981e+00]
 [-1.04077071e+00]]
rotation
3
Kalman:  [[ 1.09685254e+00]
 [ 1.44690302e-04]
 [ 8.98770690e-02]
 [ 1.01588336e+00]
 [-1.14529550e+00]]
rotation
4
Kalman:  [[ 1.06261584]
 [-0.00232417]
 [ 0.11022788]
 [ 0.99988863]
 [-1.55915923]]
rotation
5
Kalman:  [[ 1.03944629]
 [-0.00530424]
 [ 0.12905276]
 [ 0.99377781]
 [-1.17341307]]
rotation
6
Kalman:  [[ 1.03440501]
 [-0.00610932]
 [ 0.14567718]
 [ 0.95745175]
 [-0.95808274]]
rotation
7
Kalman:  [[ 1.03114449]
 [-0.00654438]
 [ 0.1620455 ]
 [ 0.97756829]
 [-1.04579314]]
rotation
8
Kalman:  [[ 1.04074742]
 [-0.005148  ]
 [ 0.17713562]
 [ 0.98525213]
 [-0.73937628]]
rotation
9
Kalman:  [[ 1.05215481]
 [-0.00303733]
 [ 0.19211216]
 [ 0.98818709]
 [-0.89427097]]
rotation
10
Ka

In [9]:
await stop_robot()